transfer learning - VGG

In [0]:
import keras
from keras.applications.vgg16 import VGG16
from keras.datasets import mnist
import numpy as np
from keras.datasets import cifar10
from keras.layers import Flatten,Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from keras.models import Model
from keras.utils import np_utils
from keras.applications.vgg16 import preprocess_input
from PIL import Image

BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-4

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# from categorical to 1-bit-hot
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# get archs and pre-trained weights
# include top to remove the classification layer
vgg = VGG16(input_shape=(32,32,3), weights='imagenet', include_top=False)
# freeze wiehgts
for layer in vgg.layers:
  layer.trainable = False
# add trainable layers 
x = GlobalAveragePooling2D()(vgg.output)
x= BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.6)(x)
prediction = Dense(10, activation='softmax')(x)
# create a model
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(lr=LEARNING_RATE),metrics=['accuracy'])


Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0  

In [0]:
# prepare data augmentation - to make more robust
# preprocess_input transforms to the needed formation of the used network
train_datagen = ImageDataGenerator(rotation_range=20,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.1,zoom_range=0.2,horizontal_flip=True,vertical_flip=True,preprocessing_function=preprocess_input)
train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train,y_train, batch_size=BATCH_SIZE)

val_datagen = ImageDataGenerator(rotation_range=20,width_shift_range=0.1,height_shift_range=0.1,shear_range=0.1,zoom_range=0.2,horizontal_flip=True,vertical_flip=True,preprocessing_function=preprocess_input)
val_datagen.fit(X_test)
val_generator = val_datagen.flow(X_test,y_test,batch_size=BATCH_SIZE)

train_steps_per_epoch = X_train.shape[0] // BATCH_SIZE
val_steps_per_epoch = X_test.shape[0] // BATCH_SIZE

history = model.fit_generator(train_generator,steps_per_epoch=train_steps_per_epoch,validation_data=val_generator,validation_steps=val_steps_per_epoch,epochs=EPOCHS,verbose=1)

Epoch 1/10
1562/1562 [==============================] - 53s 34ms/step - loss: 0.2869 - accuracy: 0.9005 - val_loss: 0.2473 - val_accuracy: 0.9092
Epoch 2/10
1562/1562 [==============================] - 52s 33ms/step - loss: 0.2472 - accuracy: 0.9061 - val_loss: 0.2072 - val_accuracy: 0.9140
Epoch 3/10
1562/1562 [==============================] - 52s 34ms/step - loss: 0.2345 - accuracy: 0.9100 - val_loss: 0.2620 - val_accuracy: 0.9173
Epoch 4/10
1562/1562 [==============================] - 52s 33ms/step - loss: 0.2271 - accuracy: 0.9129 - val_loss: 0.2413 - val_accuracy: 0.9191
Epoch 5/10
1562/1562 [==============================] - 52s 33ms/step - loss: 0.2232 - accuracy: 0.9143 - val_loss: 0.1831 - val_accuracy: 0.9203
Epoch 6/10
1562/1562 [==============================] - 53s 34ms/step - loss: 0.2200 - accuracy: 0.9147 - val_loss: 0.1484 - val_accuracy: 0.9215
Epoch 7/10
1562/1562 [==============================] - 53s 34ms/step - loss: 0.2173 - accuracy: 0.9160 - val_loss: 0.1228 -